In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from torchvision import datasets, transforms
# from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt


labels_dict = {
    0: "Camiseta/Top (T-Shirt/Top)",
    1: "Pantalón (Trouser)",
    2: "Jersey (Pullover)",
    3: "Vestido (Dress)",
    4: "Abrigo (Coat)",
    5: "Sandalia (Sandal)",
    6: "Camisa (Shirt)",
    7: "Zapatilla (Sneaker)",
    8: "Bolso (Bag)",
    9: "Botín (Ankle Boot)",
}


In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=transforms.ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=transforms.ToTensor()
)


In [3]:
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)



device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [4]:

print("The train set contains {} images, in {} batches".format(len(train_dataloader.dataset), len(train_dataloader)))
# print("The validation set contains {} images, in {} batches".format(len(valid_loader.dataset), len(valid_loader)))
print("The test set contains {} images, in {} batches".format(len(test_dataloader.dataset), len(test_dataloader)))

The train set contains 60000 images, in 938 batches
The test set contains 10000 images, in 157 batches


In [ ]:
from logging import log

class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu = nn.Sequential(
        nn.Linear(28*28, 128),
        nn.ReLU(),       
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64,10),
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu(x)
    return logits


model = Net().to(device)
print(model)

loss_fn = nn.CrossEntropyLoss()
# loss_fn = nn.SoftMarginLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


Net(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=64, out_features=10, bias=True)
    (7): Softmax(dim=None)
  )
)


In [6]:

def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()

  for batch, (X,y) in enumerate(dataloader):
    X,y = X.to(device), y.to(device)

    pred = model(X)
    loss = loss_fn(pred,y)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0,0

  with torch.inference_mode():
    for X,y in dataloader:
      X,y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [7]:
epochs = 40

for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302166 [    0/60000]


c:\Users\usuario\anaconda3\envs\houdini\Lib\site-packages\torch\nn\modules\module.py:1751: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


loss: 1.874036 [ 6400/60000]
loss: 1.756269 [12800/60000]
loss: 1.810364 [19200/60000]
loss: 1.751698 [25600/60000]
loss: 1.696662 [32000/60000]
loss: 1.697043 [38400/60000]
loss: 1.675392 [44800/60000]
loss: 1.615767 [51200/60000]
loss: 1.702729 [57600/60000]
Test Error: 
 Accuracy: 80.8%, Avg loss: 1.655128 

Epoch 2
-------------------------------
loss: 1.619919 [    0/60000]
loss: 1.685158 [ 6400/60000]
loss: 1.582764 [12800/60000]
loss: 1.696723 [19200/60000]
loss: 1.678413 [25600/60000]
loss: 1.611206 [32000/60000]
loss: 1.636673 [38400/60000]
loss: 1.702369 [44800/60000]
loss: 1.662516 [51200/60000]
loss: 1.644820 [57600/60000]
Test Error: 
 Accuracy: 83.5%, Avg loss: 1.626918 

Epoch 3
-------------------------------
loss: 1.609441 [    0/60000]
loss: 1.648451 [ 6400/60000]
loss: 1.547835 [12800/60000]
loss: 1.669425 [19200/60000]
loss: 1.654956 [25600/60000]
loss: 1.626909 [32000/60000]
loss: 1.588059 [38400/60000]
loss: 1.651268 [44800/60000]
loss: 1.635005 [51200/60000]
loss

In [9]:
# Save the model
torch.save(model.state_dict(), 'fashion_mnist_model.pth')

print('Model trained and saved.')

Model trained and saved.


In [10]:
# Load the saved model
model = Net().to(device)
model.load_state_dict(torch.load('fashion_mnist_model.pth', weights_only=True))
model.eval()

from PIL import Image, ImageOps

image_path = 'z.png'
image = Image.open(image_path).convert('L')
# image = ImageOps.invert(image_inv)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((28, 28))
])
image = transform(image).to(device)
print(f"Image shape: {image.shape}")


# Make a prediction 
with torch.no_grad():
    output = model(image)
    prediction = torch.argmax(output).item()

    print(f'Predicted digit: {prediction}')
    print(f"Fashion: {labels_dict[prediction]}")
    print("model:\n{}".format(model))
    print("dir(model):\n{}".format(dir(model)))


Image shape: torch.Size([1, 28, 28])
Predicted digit: 5
Fashion: Sandalia (Sandal)
model:
Net(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=64, out_features=10, bias=True)
    (7): LogSoftmax(dim=None)
  )
)
dir(model):
['T_destination', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_hooks', '_backward_pre_hooks